In [ ]:
# default_exp model.core

# Model

> API details. @nathan

In [ ]:
# export
import json
import logging
import torch

import pandas as pd
import tensorflow as tf

from abc import ABC, abstractmethod
from icodegen.data.core import convert_df_to_tfds, java_special_tokens, train_tokenizer
from pathlib import Path, PurePath
from tokenizers import Tokenizer

import datetime

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
# export
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM, TFGPT2LMHeadModel, GPT2Config, GPT2Tokenizer
import time

from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import pandas as pd

from icodegen.data.core import convert_df_to_tfds, java_special_tokens, train_tokenizer, replace_spec_toks_to_original
from typing import Optional, Dict

In [ ]:
# Setup

df_fake = pd.DataFrame(
    ["aaaa(bb(aaaa(bb()()ccc)dd)()ccc)dd", "aaaa(bb()ccccc)dd"], columns=["code"]
)

# Tokenize the data
max_length = 16
batch_size = 1
vocab_sz = 100
tokenizer = train_tokenizer(df_fake, java_special_tokens, max_length, vocab_sz=vocab_sz)
dataset = convert_df_to_tfds(df_fake, tokenizer, max_length, batch_size)

Load Codesearchnet Java

In [ ]:
java_df = pd.read_csv('/tf/main/dvc-icodegen/data/clean_java.csv')

In [ ]:
java_df.head()

In [ ]:
java_df.describe()

In [ ]:
java_df.shape

In [ ]:
java_train = java_df[java_df['partition'] == 'train']

In [ ]:
java_train.shape

In [ ]:
java_trn_samples = java_train.sample(100000)

In [ ]:
# Tokenize the data

max_length = 112
batch_size = 16
vocab_sz = 10000
tokenizer = train_tokenizer(java_trn_samples, java_special_tokens, max_length, vocab_sz=vocab_sz)

In [ ]:
java_trn_dataset = convert_df_to_tfds(java_trn_samples, tokenizer, max_length, batch_size)

In [ ]:
java_trn_dataset

In [ ]:
tokenizer.get_vocab_size()

In [ ]:
# export
def _loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(
        labels, logits, from_logits=True
    )

In [ ]:
# export
class Model(ABC):
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model

    @abstractmethod
    def from_path(path):
        pass

    @abstractmethod
    def get_probs(self, inputs):
        pass

    @abstractmethod
    def save(self, path):
        pass

    @abstractmethod
    def tokenize(self, method):
        pass

    @abstractmethod
    def train(self, ds, epochs):
        pass

# Transformer Models

HuggingFace-based implementation, originally [this](!https://huggingface.co/transformers/model_doc/gpt2.html).

In [ ]:
# export

class TransformerHFModel(Model):
    def __init__(
        self,
        out_path,
        model,
        tokenizer,
        optimizer,
        loss,
        name_sufix=None
    ):
        """
        Constructs a custom transformer model using a provided HF model
        :param out_path: 
        :param model: HF transformer model
        :tokenizer: HF tokenizer model
        :optimizer: Keras optimizer
        :loss: Custom loss function for model training
        """
        
        # HuggingFace model implementation is left outside this class in order to
        # decouple from any specific implementation, besides aiming experimentation with multiple models
        
        model_config = model.config
        self.config_name = f"tfr_layers{model_config.n_layer}_vocab{model_config.vocab_size}_embd{model_config.n_embd}_heads{model_config.n_head}"
        
        if name_sufix is not None:
            self.config_name += f"_{name_sufix}"
        
        pure_path = PurePath(str(out_path))
        
        if pure_path.name != self.config_name:
            self.out_path = Path(out_path) / self.config_name
        else:
            self.out_path = Path(out_path)
        
        self.out_path.mkdir(exist_ok=True)
        self.optimizer = optimizer
        self.loss = loss
        
        tensorboard_path = self.out_path / "tensorboard_logs"
        tensorboard_path.mkdir(exist_ok=True)
        tensorboard_path = tensorboard_path  / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        
        self.callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                filepath=self.out_path / "ckpt_{epoch}", save_weights_only=True
            ),
            tf.keras.callbacks.TensorBoard(
                log_dir=str(tensorboard_path),
                histogram_freq=0,  # How often to log histogram visualizations
                embeddings_freq=0,  # How often to log embedding visualizations
                update_freq="epoch",
            ),  # How often to write logs (default: once per epoch)
            tf.keras.callbacks.EarlyStopping(
                # Stop training when `val_loss` is no longer improving
                monitor="val_loss",
                # "no longer improving" being defined as "no better than 1e-2 less"
                min_delta=1e-2,
                # "no longer improving" being further defined as "for at least 5 epochs"
                patience=5,
                verbose=1,
            ),
        ]
        
        super().__init__(tokenizer, model)
        
    @staticmethod
    def from_path(model_path, hf_model, optimizer, loss):
        path = Path(model_path)
        
        # Load tokenizer
        tokenizer = Tokenizer.from_file(str(path / "tokenizer.json"))
        
        # Load model
        with open(path / "model_config.json", "r") as f:
            model_config = json.load(f)
        full_path = path / "model"
        loaded_model = hf_model.from_pretrained(full_path)
        
        name_sufix = None
        if "_pretrained" in str(full_path):
            name_sufix = "pretrained"
        
        model = TransformerHFModel(model_path, loaded_model, tokenizer, optimizer, loss, name_sufix=name_sufix)
        
        return model
    
    def save(self):
        # Save tokenizer
        self.tokenizer.save(str(self.out_path / "tokenizer.json"), pretty=True)
        # Save HF model
        self.model.save_pretrained(self.out_path / "model")
        
        # Save model config.
        model_config = self.model.config
        # TODO: extract attributes from configuration
        config_dict = {
          "activation_function": model_config.activation_function,
          "attn_pdrop": model_config.attn_pdrop,
          "bos_token_id": model_config.bos_token_id,
          "embd_pdrop": model_config.embd_pdrop,
          "eos_token_id": model_config.eos_token_id,
          "gradient_checkpointing": model_config.gradient_checkpointing,
          "initializer_range": model_config.initializer_range,
          "layer_norm_epsilon": model_config.layer_norm_epsilon,
          "model_type": model_config.model_type,
          "n_ctx": model_config.n_ctx,
          "n_embd": model_config.n_embd,
          "n_head": model_config.n_head,
          "n_inner": model_config.n_inner,
          "n_layer": model_config.n_layer,
          "n_positions": model_config.n_positions,
          "pad_token_id": model_config.pad_token_id,
          "resid_pdrop": model_config.resid_pdrop,
          "summary_activation": model_config.summary_activation,
          "summary_first_dropout": model_config.summary_first_dropout,
          "summary_proj_to_labels": model_config.summary_proj_to_labels,
          "summary_type": model_config.summary_type,
          "summary_use_proj": model_config.summary_use_proj,
          "vocab_size": model_config.vocab_size
        }
        
        with open(self.out_path / "model_config.json", "w") as file:
            json.dump(config_dict, file)
        
    
    def generate(self, max_length, n, top_k=10, temperature=1.0):
        """
        Performs generation process through sampling strategy (top-k) by default
        :param max_length: Max. allowed length for generated sequences
        :param n: Number of samples to generate
        :param top_k: top-k most likely next words to be filtered for the sampling
        :returns Generated sequences:
        """
        
        input_seed = [self.tokenizer.encode("<sos>").ids[0]]
        input_ids = tf.expand_dims(input_seed, 0)
        
        gen_txt = self.model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            top_k=top_k,
            num_return_sequences=n
        )
        
        generated = []
        for i, sample in enumerate(gen_txt):
            generated.append(self.tokenizer.decode(sample, skip_special_tokens=False))
            
        return generated
    
    def get_probs(self, inputs):
        outputs = self.model(inputs)
        logits = outputs[0]
        probs = tf.nn.softmax(logits)

        return probs    
    
    def sample_top_k(self, max_length, top_k=50):
        """
        Performs sampling (generation) using top-k mechanism
        :param max_length: Max. length allowed in the generation process
        :param top_k: top-k most likely next words to be filtered for the sampling
        :returns: Generated text
        """
        input_seed = [self.tokenizer.encode("<sos>").ids[0]]
        input_ids = tf.expand_dims(input_seed, 0)
        
        gen_txt = self.model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            top_k=top_k
        )
        
        return self.tokenizer.decode(gen_txt[0], skip_special_tokens=False)
    
    def sample_nucleus(self, max_length, top_p=0.9):
        """
        Perform sampling using top-p (nucleus) mechanism
        :param  max_length: Max. allowed length of generated sequences 
        :param top_p: Probability value to find possible set of words whose cumulative probability exceeds top-p value
        :returns: Generated text
        """
        input_seed = [self.tokenizer.encode("<sos>").ids[0]]
        input_ids = tf.expand_dims(input_seed, 0)

        gen_txt = self.model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            top_p=top_p,
            top_k=0
        )

        return self.tokenizer.decode(gen_txt[0], skip_special_tokens=False)
    
    def sample_temperature(self, max_length, temperature=0.7):
        """
        Performs sampling using temperature strategy
        :param max_length: Max. allowed length of generated sequences 
        :param temperature: Temperature parameter of the softmax
        :returns: Generated text
        """
        input_seed = [self.tokenizer.encode("<sos>").ids[0]]
        input_ids = tf.expand_dims(input_seed, 0)

        gen_txt = self.model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            top_k=0,
            temperature=temperature
        )
        
        return self.tokenizer.decode(gen_txt[0], skip_special_tokens=False)
        

    def tokenize(self, method):
        """
        :param method: Code snippet (plain text)
        :returns: Encoded result using the provided tokenizer
        """
        output = {}
        # encode method and then convert to format that hf models expect
        encoding = self.tokenizer.encode("<sos>" + method)
        output["input_ids"] = tf.expand_dims(
            tf.convert_to_tensor(encoding.ids, dtype=tf.int32), 0
        )
        output["attention_mask"] = tf.expand_dims(
            tf.convert_to_tensor(encoding.attention_mask, dtype=tf.int32), 0
        )

        return output

        # return self.tokenizer(method, return_tensors="tf")

    def train(self, ds_train, ds_val, epochs):
        """
        Performs training process leveraging keras methods.
        :param ds_train: Tensorflow dataset for training
        :param ds_val: Tensorflow dataset for validation
        :param epochs: Number of epochs to execute training
        """
        # Be careful with the definition of the loss function when compiling
        # Gotta consider the number of layers (n_layer param.) --> actually number of decoder blocks
        self.model.compile(optimizer=self.optimizer, loss=[self.loss, *[None] * self.model.config.n_layer])
        history = self.model.fit(ds_train, epochs=epochs, callbacks=self.callbacks, validation_data=ds_val)
        
        return history

Test GPT2 - HuggingFace-based

In [ ]:
# Establish configuration for HF model, in this case by means of GPT2Config class

custom_config_gpt2 = GPT2Config(
    vocab_size = tokenizer.get_vocab_size(),
    bos_token_id = tokenizer.token_to_id("<sos>"),
    eos_token_id = tokenizer.token_to_id("<eos>"),
    pad_token_id = tokenizer.token_to_id("<pad>")
)

In [ ]:
# Configuration

gpt2_hf_model_path = "models/GPT2-HF"

In [ ]:
# Implement the corresponding HuggingFace model using the custom configuration

hf_gpt_model = TFGPT2LMHeadModel(custom_config_gpt2)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)

In [ ]:
gpt2_model = TransformerHFModel(gpt2_hf_model_path, hf_gpt_model, tokenizer, optimizer, _loss)

In [ ]:
TRAIN_EPOCHS = 20

In [ ]:
start_trn_time = time.time()
gpt2_model.train(dataset, dataset, epochs=TRAIN_EPOCHS)
end_trn_time = time.time()
print(f'Time elapsed training: {end_trn_time - start_trn_time} seconds.')

In [ ]:
NUMBER_GEN_SEQS = 10

In [ ]:
start_gen_time = time.time()
generated_seqs = gpt2_model.generate(n=NUMBER_GEN_SEQS, max_length=max_length)
end_gen_time = time.time()
print(f'Time elapsed generating: {end_gen_time - start_gen_time} seconds.')

In [ ]:
for seq in generated_seqs:
    print(f'sample: {seq}')

In [ ]:
gpt2_model.save()

In [ ]:
loaded_gpt2_model = TransformerHFModel.from_path("models/GPT2-HF/tfgp_t2lm_head_model_4-20210330-171318", TFGPT2LMHeadModel, optimizer, _loss)

In [ ]:
type(loaded_gpt2_model)

In [ ]:
generated_seqs = gpt2_model.generate(n=10, max_length=max_length)
print(generated_seqs)

In [ ]:
text = "test"
encoded_text = tokenizer.encode("<sos>" + text).ids
input_eval = tf.expand_dims(encoded_text, 0)
logits = gpt2_model.model(input_eval)[0].numpy()
inputs = gpt2_model.tokenize(text)
probs = gpt2_model.get_probs(inputs)[0].numpy()

In [ ]:
probs.shape

In [ ]:
# for i in range(len(probs)):
#     assert np.isclose(1.0, probs[i].sum())
#     assert np.argmax(logits[i]) == np.argmax(probs[i])

In [ ]:
gpt2_model.sample_nucleus(max_length=20)

In [ ]:
gpt2_model.sample_top_k(max_length=20, top_k=10)

Keras-based implementation (based on [this example](!https://keras.io/examples/generative/text_generation_with_miniature_gpt/#prepare-the-data-for-wordlevel-language-modelling))

Auxiliar classes / methods for building the model.

In [ ]:
# export

def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    """
    Mask the upper half of the dot product matrix in self attention.
    This prevents flow of information from future tokens to current token.
    1's in the lower triangle, counting from the lower right corner.
    """
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)

In [ ]:
# export

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate
        
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads, embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)
    
    def get_config(self):
        return {
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "rate": self.rate
        }
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)
    
    
# export
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate
        
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads, embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output = self.att(inputs, inputs, attention_mask=causal_mask)
        attention_output = self.dropout1(attention_output)
        out1 = self.layernorm1(inputs + attention_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)
    
    def get_config(self):
        return {
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
            "rate": self.rate
        }
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
# export

class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.maxlen = maxlen
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
    def get_config(self):
        return {
            "maxlen": self.maxlen,
            "vocab_size": self.vocab_size,
            "embed_dim": self.embed_dim
        }
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [ ]:
# export

class MiniatureGPTModel(Model):
    def __init__(
        self,
        out_path, 
        max_length,
        batch_size,
        ff_dim,
        embedding_dim,
        n_heads,
        n_transformer_blocks,
        vocab_size,
        tokenizer,
        optimizer,
        loss
    ):
        """
        Builds a transformer model using tf-keras API
        :param out_path: Path for saving model data
        :param max_length: Max. length allowed for the sequences
        :param batch_size: 
        :param ff_dim:
        :param embedding_dim:
        :param n_heads:
        :param n_transformer_blocks:
        :param vocab_size:
        :param tokenizer:
        :param optimizer:
        :param loss:
        """
        self.max_length = max_length
        self.batch_size = batch_size
        self.ff_dim = ff_dim
        self.embedding_dim = embedding_dim
        self.n_heads = n_heads
        self.n_transformer_blocks = n_transformer_blocks
        self.vocab_size = vocab_size
        self.loss = loss
        self.optimizer = optimizer
        
        self.config_name = f"min_tfr_layers{n_transformer_blocks}_vocab{tokenizer.get_vocab_size()}_embd{embedding_dim}_heads{n_heads}"
        
        model = self.__create_model()
        
        model._name = "Miniature-GPT"
        
        pure_path = PurePath(str(out_path))
        
        if pure_path.name != self.config_name:
            self.out_path = Path(out_path) / self.config_name
        else:
            self.out_path = Path(out_path)
        
        self.out_path.mkdir(exist_ok=True)
        
        tensorboard_path = self.out_path / "tensorboard_logs"
        tensorboard_path.mkdir(exist_ok=True)
        tensorboard_path = tensorboard_path / datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        self.callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                filepath=self.out_path / "ckpt_{epoch}", save_weights_only=True
            ),
            tf.keras.callbacks.TensorBoard(
                log_dir=str(tensorboard_path),
                histogram_freq=0,  # How often to log histogram visualizations
                embeddings_freq=0,  # How often to log embedding visualizations
                update_freq="epoch",
            ),  # How often to write logs (default: once per epoch)
            tf.keras.callbacks.EarlyStopping(
                # Stop training when `val_loss` is no longer improving
                monitor="val_loss",
                # "no longer improving" being defined as "no better than 1e-2 less"
                min_delta=1e-2,
                # "no longer improving" being further defined as "for at least 5 epochs"
                patience=5,
                verbose=1,
            ),
        ]

        super().__init__(tokenizer, model)
     
    def __create_model(self):
        inputs = layers.Input(shape=(self.max_length-1,), dtype=tf.int32)
        embedding_layer = TokenAndPositionEmbedding(self.max_length-1, self.vocab_size, self.embedding_dim)
        x = embedding_layer(inputs)
        transformerLayers = []
        outputs_x = []
        for i in range(self.n_transformer_blocks):
            transformerLayers.append(TransformerBlock(self.embedding_dim, self.n_heads, self.ff_dim))

        for i in range(self.n_transformer_blocks):
            x = transformerLayers[i](x)
            outputs_x.append(x)
        # transformer_block = TransformerBlock(self.embedding_dim, self.n_heads, self.ff_dim)
        # x = transformer_block(x)

        outputs = layers.Dense(self.vocab_size)(outputs_x[self.n_transformer_blocks-1])
        # outputs = layers.Dense(self.vocab_size)(x)
        model = keras.Model(inputs=inputs, outputs=[outputs, *outputs_x])
        
        return model
        
    @staticmethod
    def from_path(path, optimizer, loss):
        path = Path(path)
        # Load tokenizer
        tokenizer = Tokenizer.from_file(str(path / "tokenizer.json"))
        # Load model
        with open(path / "model_config.json", "r") as f:
            model_config = json.load(f)
            
        min_model = MiniatureGPTModel(
            path,
            **model_config,
            tokenizer=tokenizer,
            optimizer=optimizer, 
            loss=loss
        )
        
        loaded_model = tf.keras.models.load_model(str(path), custom_objects={"_loss": _loss})
        min_model.model = loaded_model
        
        return min_model
        
    def save(self):
        # Save tokenizer
        self.tokenizer.save(str(self.out_path / "tokenizer.json"), pretty=True)
        # Save model
        self.model.save(str(self.out_path))
        # Save config.
        model_config = {
            "max_length": self.max_length,
            "batch_size": self.batch_size,
            "ff_dim": self.ff_dim,
            "embedding_dim": self.embedding_dim,
            "n_heads": self.n_heads,
            "n_transformer_blocks": self.n_transformer_blocks,
            "vocab_size": self.vocab_size,
        }
        with open(self.out_path / "model_config.json", "w") as file:
            json.dump(model_config, file)
        
    def tokenize(self, method):
        """
        :param method: Code snippet
        :returns: Encoded result using the provided tokenizer
        """
        output = {}
        # encode method and then convert to format that hf models expect
        encoding = self.tokenizer.encode("<sos>" + method)
        output["input_ids"] = tf.expand_dims(
            tf.convert_to_tensor(encoding.ids, dtype=tf.int32), 0
        )
        output["attention_mask"] = tf.expand_dims(
            tf.convert_to_tensor(encoding.attention_mask, dtype=tf.int32), 0
        )

        return output
    
    def generate(self, n, top_k=50):
        """Performs generation process through top-k sampling process"""
         # Converting our start string to numbers (vectorizing)
        start_tokens = [self.tokenizer.encode("<sos>").ids[0]]
        
        generated_txt = []
        
        while len(generated_txt) < n:
            gen_sample = self.__sample_top_k(start_tokens, top_k)
            generated_txt.append(gen_sample)
        
        txt_seqs = []
        for seq in generated_txt:
            txt_seqs.append(self.tokenizer.encode(seq))
            
        return txt_seqs
        
    def __sample_top_k(self, start_tokens, top_k=10):
        num_tokens_generated = 0
        tokens_generated = [] 
        gen_sequence = start_tokens[:]
        while len(gen_sequence) <= self.max_length-1:
            pad_len = (self.max_length-1) - len(gen_sequence)
            sample_index = len(gen_sequence) - 1
            if pad_len < 0:
                x = gen_sequence[:self.max_length-1]
                sample_index = self.max_length - 1
            elif pad_len > 0:
                x = gen_sequence + [0] * pad_len
            else:
                x = gen_sequence[:]
            sample_index = len(gen_sequence)-1
            x = np.array([x])
            prediction_output = self.model.predict(x)
            y = prediction_output[0]
            sample_token = self.__sample_from(y[0][sample_index], top_k)
            tokens_generated.append(sample_token)
            gen_sequence.append(sample_token)
            num_tokens_generated = len(tokens_generated)
            
        return tokens_generated
            
    def __sample_from(self, logits, top_k):
        logits, indices = tf.math.top_k(logits, k=top_k, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)
    
    def get_probs(self, inputs):
        #         ids = self.tokenizer.encode("<sos>" + method).ids
        #         input_eval = tf.expand_dims(ids, 0)

        logits = self.model(inputs["input_ids"])
        probs = tf.nn.softmax(logits)  # [0].numpy()

        return probs
        
    def train(self, ds_train, ds_val, epochs):
        """
        Performs training process leveraging keras methods
        """
        self.model.compile(optimizer=self.optimizer, loss=[self.loss, *[None]* self.n_transformer_blocks])
        history = self.model.fit(ds_train, epochs=epochs, callbacks=self.callbacks, validation_data=ds_val)
        
        return history

Test Miniature GPT. Keras-based

In [ ]:
# Configuration parameters
# 
min_gpt_config_params = {
    "out_path": "models/Min-GPT",
    "max_length" : max_length,
    "batch_size": batch_size,
    "ff_dim": 64,
    "embedding_dim": 128,
    "n_heads": 2,
    "n_transformer_blocks": 1,
    "vocab_size": tokenizer.get_vocab_size()
}

In [ ]:
# Optimizer, loss definition

min_gpt_optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
min_gpt_loss = _loss

In [ ]:
min_gpt_config_params["optimizer"] = min_gpt_optimizer
min_gpt_config_params["loss"] = min_gpt_loss
min_gpt_config_params["tokenizer"] = tokenizer

In [ ]:
min_gpt_model = MiniatureGPTModel(**min_gpt_config_params)

In [ ]:
start_trn_min_time = time.time()
min_gpt_history = min_gpt_model.train(dataset, dataset, epochs=TRAIN_EPOCHS)
end_trn_min_time = time.time()
print(f'Time elapsed: {end_trn_min_time - start_trn_min_time} seconds')

In [ ]:
dataset

In [ ]:
min_gpt_model.model.summary()

In [ ]:
gen_sequences = min_gpt_model.generate(n=10)

In [ ]:
for seq in gen_sequences:
    print('-'*100)
    print(f'seq: {tokenizer.decode(seq, skip_special_tokens=False)}')

In [ ]:
min_gpt_model.save()

In [ ]:
loaded_min_gpt = MiniatureGPTModel.from_path("models/Min-GPT/Miniature-GPT-20210330-200608", optimizer, min_gpt_loss)

In [ ]:
type(loaded_min_gpt)

In [ ]:
# export
class RNNModel(Model):
    _RNN_TYPE = {
        "rnn": tf.keras.layers.SimpleRNN,
        "gru": tf.keras.layers.GRU,
        "lstm": tf.keras.layers.LSTM,
    }

    def __init__(
        self,
        rnn_type,
        n_layers,
        vocab_size,
        embedding_dim,
        rnn_units,
        batch_size,
        out_path,
        tokenizer,
    ):
        self.rnn_type = rnn_type
        self.n_layers = n_layers
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.rnn_units = rnn_units

        self.config_name = f"{rnn_type}_layers{n_layers}_vocab{vocab_size}_embed{embedding_dim}_units{rnn_units}"
        self.out_path = Path(out_path) / self.config_name
        self.out_path.mkdir(exist_ok=True)
        tensorboard_path = self.out_path / "tensorboard_logs"
        tensorboard_path.mkdir(exist_ok=True)
        self.callbacks = [
            tf.keras.callbacks.ModelCheckpoint(
                filepath=self.out_path / "ckpt_{epoch}", save_weights_only=True
            ),
            tf.keras.callbacks.TensorBoard(
                log_dir=str(tensorboard_path),
                histogram_freq=0,  # How often to log histogram visualizations
                embeddings_freq=0,  # How often to log embedding visualizations
                update_freq="epoch",
            ),  # How often to write logs (default: once per epoch)
            tf.keras.callbacks.EarlyStopping(
                # Stop training when `val_loss` is no longer improving
                monitor="val_loss",
                # "no longer improving" being defined as "no better than 1e-2 less"
                min_delta=1e-2,
                # "no longer improving" being further defined as "for at least 5 epochs"
                patience=5,
                verbose=1,
            ),
        ]

        layer = RNNModel._RNN_TYPE[rnn_type]
        rnn_layers = [
            layer(
                rnn_units,
                return_sequences=True,
                recurrent_initializer="glorot_uniform",
                # following BigCode != Big Vocab Paper
                dropout=0.5,
            )
            for _ in range(n_layers)
        ]
        model = tf.keras.Sequential(
            [
                tf.keras.layers.Embedding(
                    input_dim=vocab_size,
                    output_dim=embedding_dim,
                    mask_zero=True,  # Zero cannot be used in the vocabulary
                ),
            ]
            + rnn_layers
            + [
                tf.keras.layers.Dense(vocab_size),
            ]
        )

        super().__init__(tokenizer, model)

    @staticmethod
    def from_path(path):
        path = Path(path)

        tokenizer = Tokenizer.from_file(str(path / "tokenizer.json"))
        with open(path / "model_config.json", "r") as f:
            model_config = json.load(f)

        model = RNNModel(
            model_config["rnn_type"],
            model_config["n_layers"],
            model_config["vocab_size"],
            model_config["embedding_dim"],
            model_config["rnn_units"],
            1,
            path,
            tokenizer,
        )
        model.model = tf.keras.models.load_model(
            str(path), custom_objects={"_loss": _loss}
        )

        return model

    def get_probs(self, inputs):
        #         ids = self.tokenizer.encode("<sos>" + method).ids
        #         input_eval = tf.expand_dims(ids, 0)

        logits = self.model(inputs)
        probs = tf.nn.softmax(logits)  # [0].numpy()

        return probs

    def generate(self, n, temperature=1.0):
        # Converting our start string to numbers (vectorizing)
        text_generated = [self.tokenizer.encode("<sos>").ids[0]]
        input_eval = tf.expand_dims(text_generated, 0)

        # Here batch size == 1
        self.model.reset_states()
        for i in range(n):
            predictions = self.model(input_eval)
            # remove the batch dimension
            predictions = tf.squeeze(predictions, 0)

            # using a categorical distribution to predict the character
            # returned by the model
            predictions = predictions / temperature
            predicted_id = tf.random.categorical(predictions, num_samples=1)[
                -1, 0
            ].numpy()

            text_generated.append(predicted_id)
            # Pass the predicted character as the next input to the model
            # along with the previous hidden state
            input_eval = tf.expand_dims(text_generated, 0)

        return self.tokenizer.decode(text_generated, skip_special_tokens=False)

    def save(self):
        self.tokenizer.save(str(self.out_path / "tokenizer.json"), pretty=True)
        self.model.save(str(self.out_path))
        model_config = {
            "rnn_type": self.rnn_type,
            "n_layers": self.n_layers,
            "vocab_size": self.vocab_size,
            "embedding_dim": self.embedding_dim,
            "rnn_units": self.rnn_units,
        }
        with open(self.out_path / "model_config.json", "w") as f:
            json.dump(model_config, f)

    def tokenize(self, method):
        #         ids = self.tokenizer.encode("<sos>" + method).ids
        #         inputs = tf.expand_dims(ids, 0)
        output = {}
        # encode method and then convert to format that hf models expect
        encoding = self.tokenizer.encode("<sos>" + method)
        output["input_ids"] = tf.expand_dims(
            tf.convert_to_tensor(encoding.ids, dtype=tf.int32), 0
        )
        output["attention_mask"] = tf.expand_dims(
            tf.convert_to_tensor(encoding.attention_mask, dtype=tf.int32), 0
        )
        return output  # self.tokenizer(method, return_tensors="tf")

    # TODO add tensorboard call back for easy visualization
    def train(self, ds_trn, ds_val, epochs):
        self.model.compile(optimizer="adam", loss=_loss)
        history = self.model.fit(
            ds_trn, epochs=epochs, callbacks=self.callbacks, validation_data=ds_val
        )

        return history

# RNN models

In [ ]:
data_path = Path("/tmp/data/codesearchnet")
df_trn = pd.read_json(
    data_path / "codesearchnet_java" / "train.jsonl",
    orient="records",
    lines=True,
)[:50]
ds_trn = convert_df_to_tfds(df_trn, tokenizer, max_length, batch_size)

In [ ]:
tok_trn = tokenizer.encode_batch(df_trn.code.values)
hf_fast_tok_form = {
    "input_ids": [x.ids for x in tok_trn],
    "attention_mask": [x.attention_mask for x in tok_trn],
}

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (dict(train_encodings), train_labels)
)

In [ ]:
tokenizer.encode("hi")

Encoding(num_tokens=16, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
tokenizer.encode_batch()

In [ ]:
tok_path = Path("/tmp/models/tokenizer.json")

In [ ]:
from transformers import GPT2TokenizerFast, PreTrainedTokenizerFast

new_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

In [ ]:
new_tokenizer(["Hi there", "WasaaP"])

{'input_ids': [[17250, 612], [16973, 7252, 47]], 'attention_mask': [[1, 1], [1, 1, 1]]}

In [ ]:
GPT2TokenizerFast()

TypeError: __init__() missing 2 required positional arguments: 'vocab_file' and 'merges_file'

In [ ]:
new_tokenizer.backend_tokenizer = tokenizer

AttributeError: can't set attribute

In [ ]:
from transformers import TFDistilBertForSequenceClassification

model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(
    optimizer=optimizer, loss=model.compute_loss
)  # can also use any keras loss fn
# model.fit(train_dataset.shuffle(1000).batch(16), epochs=3, batch_size=16)

In [ ]:
# export
class WildModel(Model):
    def __init__(
        self,
        model_name,
    ):
        tokenizer = AutoTokenizer.from_pretrained(model_name, pad_token="<|endoftext|>")
        model = AutoModelForCausalLM.from_pretrained(model_name, pad_token_id=50256)
        super().__init__(tokenizer, model)

    @staticmethod
    def from_path(path):
        pass

    def get_probs(self, inputs):
        # inputs = torch.from_numpy(inputs)
        logits = self.model(**inputs).logits
        probs = F.softmax(logits)

        return probs

    def generate(self, n, temperature=1.0):
        pass

    def save(self):
        pass

    def batch_tokenize(self, batch):
        return self.tokenizer(batch, padding=True, truncation=True, return_tensors='pt')

    def tokenize(self, method):
        pass

    # TODO add tensorboard call back for easy visualization
    def train(self, ds_trn, ds_val, epochs):
        pass

In [ ]:
PARAM_COUNT = 124_772

rnn_type = "gru"
n_layers = 1
vocab_size = tokenizer.get_vocab_size()
embedding_dim = 128
rnn_units = 128
batch_size = 1
out_path = "/tmp"
gru = RNNModel(
    rnn_type,
    n_layers,
    vocab_size,
    embedding_dim,
    rnn_units,
    batch_size,
    out_path,
    tokenizer,
)

assert PARAM_COUNT == gru.model.count_params()

In [ ]:
gpt2_configuration = GPT2Config()

In [ ]:
TFLM

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

In [ ]:
gpt2_model = TransformerModel('gpt2_model', , tokenizer, optimizer, _loss)

In [ ]:
transformer_history = gpt2_model.train(dataset, dataset, EPOCHS)

In [ ]:
gpt2_model.generate(10, 10)

In [ ]:
EPOCHS = 1
chkpt_path = Path(out_path) / (
    f"{rnn_type}_layers{n_layers}_vocab{vocab_size}_embed{embedding_dim}_units{rnn_units}"
)
history = gru.train(dataset, dataset, EPOCHS)

assert chkpt_path.exists()
assert EPOCHS == len(list(chkpt_path.glob("*.index")))
assert EPOCHS == len(history.history["loss"])

In [ ]:
# How do I test this?
text = "test"
text_generated = tokenizer.encode("<sos>" + text).ids
input_eval = tf.expand_dims(text_generated, 0)

logits = gru.model(input_eval)[0].numpy()
inputs = gru.tokenize(text)
probs = gru.get_probs(inputs)[0].numpy()

for i in range(len(probs)):
    assert np.isclose(1.0, probs[i].sum())
    assert np.argmax(logits[i]) == np.argmax(probs[i])

In [ ]:
# Add test case for earlystopping

In [ ]:
# # this test will break sometimes due to encoding adding space prefixes to some
# # tokens when add_prefix_space=False it should be fine :/
# NUM_TOKENS = 10
# text = gru.generate(NUM_TOKENS)
# tokenizer.no_padding()
# ids = tokenizer.encode(text).ids
# tokenizer.enable_padding(length=max_length)

# # -1 for the <sos> token that's always prepended
# assert NUM_TOKENS == len(ids) - 1

In [ ]:
gru.save()

assert (gru.out_path / "assets").exists()
assert (gru.out_path / "model_config.json").exists()
assert (gru.out_path / "saved_model.pb").exists()
assert (gru.out_path / "tokenizer.json").exists()
assert (gru.out_path / "variables").exists()

In [ ]:
loaded_gru = RNNModel.from_path(str(gru.out_path))

assert gru.tokenizer.get_vocab() == loaded_gru.tokenizer.get_vocab()
assert gru.model.count_params() == loaded_gru.model.count_params()
assert gru.model.evaluate(dataset, verbose=2) == loaded_gru.model.evaluate(
    dataset, verbose=2
)

In [ ]:
# export

# Experiment 0.0.0
VANILLA_CONFIG = {
    "rnn_type": "rnn",
    "n_layers": 1,
    "embedding_dim": 256,
    "rnn_units": 1_024,
}

# Experiment 1.0.0
GRU_CONFIG_1 = {
    "rnn_type": "gru",
    "n_layers": 1,
    "embedding_dim": 256,
    "rnn_units": 1_024,
}

# Experiment 1.1.0
GRU_CONFIG_2 = {
    "rnn_type": "gru",
    "n_layers": 2,
    "embedding_dim": 256,
    "rnn_units": 1_024,
}

# Experiment 1.1.1
GRU_CONFIG_3 = {
    "rnn_type": "gru",
    "n_layers": 3,
    "embedding_dim": 256,
    "rnn_units": 1_024,
}

# Experiment 1.2.0
GRU_CONFIG_4 = {
    "rnn_type": "gru",
    "n_layers": 1,
    "embedding_dim": 256,
    "rnn_units": 512,
}

# Experiment 1.2.1
GRU_CONFIG_5 = {
    "rnn_type": "gru",
    "n_layers": 1,
    "embedding_dim": 256,
    "rnn_units": 2_048,
}

In [ ]:
# export
def train(
    data_path, out_path, epochs=64, max_length=300, batch_size=64, configs=[], n=None
):
    """Function for training models related to the library."""
    out_path.mkdir(exist_ok=True)

    # Load in the datasets

    # Train BPE tokenizer
    # Check if the path where the tokenizer is to be saved is not empty
    # if it is not empty then just load the tokenizer there.
    if (out_path / "tokenizer.json").exists():
        logging.info(f"Loading tokenizer from {str(out_path / 'tokenizer.json')}.")
        tokenizer = Tokenizer.from_file(str(out_path / "tokenizer.json"))
    else:
        logging.info(
            f"Training new tokenizer and saving to {str(out_path / 'tokenizer.json')}."
        )
        df_bpe = pd.read_json(
            data_path / "codesearchnet_java" / "bpe.jsonl", orient="records", lines=True
        )[:n]
        tokenizer = train_tokenizer(df_bpe, java_special_tokens, max_length)
        tokenizer.save(str(out_path / "tokenizer.json"), pretty=True)
        del df_bpe

    # Tokenize the dataset and convert it to tfds.
    tfds_trn_path = (
        data_path / "codesearchnet_java" / f"tfds_trn_{max_length}len_{batch_size}bs"
    )
    if tfds_trn_path.exists():
        ds_trn = tf.data.experimental.load(
            str(tfds_trn_path),
            (
                tf.TensorSpec(shape=(batch_size, max_length - 1), dtype=tf.int32),
                tf.TensorSpec(shape=(batch_size, max_length - 1), dtype=tf.int32),
            ),
        )
    else:
        df_trn = pd.read_json(
            data_path / "codesearchnet_java" / "train.jsonl",
            orient="records",
            lines=True,
        )[:n]
        ds_trn = convert_df_to_tfds(df_trn, tokenizer, max_length, batch_size)
        tfds_trn_path.mkdir(exist_ok=True)
        tf.data.experimental.save(ds_trn, str(tfds_trn_path))
        del df_trn

    tfds_val_path = (
        data_path / "codesearchnet_java" / f"tfds_val_{max_length}len_{batch_size}bs"
    )
    if tfds_val_path.exists():
        ds_val = tf.data.experimental.load(
            str(tfds_val_path),
            (
                tf.TensorSpec(shape=(batch_size, max_length - 1), dtype=tf.int32),
                tf.TensorSpec(shape=(batch_size, max_length - 1), dtype=tf.int32),
            ),
        )
    else:
        df_val = pd.read_json(
            data_path / "codesearchnet_java" / "valid.jsonl",
            orient="records",
            lines=True,
        )[:n]
        ds_val = convert_df_to_tfds(df_val, tokenizer, max_length, batch_size)
        tfds_val_path.mkdir(exist_ok=True)
        tf.data.experimental.save(ds_val, str(tfds_val_path))
        del df_val

    logging.info("Starting the training of all RNN based models.")
    # Train RNN based models
    for config in configs:
        rnn_model = RNNModel(
            config["rnn_type"],
            config["n_layers"],
            tokenizer.get_vocab_size(),
            config["embedding_dim"],
            config["rnn_units"],
            batch_size,
            str(out_path),
            tokenizer,
        )
        rnn_model.train(ds_trn, ds_val, epochs)
        rnn_model.save()

    logging.info("Starting the training of all Transformer based models.")
    # Train Transformer models
    pass

In [ ]:
data_path = Path("/tmp/data/codesearchnet")
models_path = Path("/tmp/models")
rnn_path = models_path / "rnn_layers1_vocab10000_embed256_units1024"
EPOCHS = 2
MAX_LEN = 100
BS = 8
N = 100
_RNN_CONFIGs = [VANILLA_CONFIG]

train(
    data_path=data_path,
    out_path=models_path,
    epochs=EPOCHS,
    max_length=MAX_LEN,
    batch_size=BS,
    configs=_RNN_CONFIGs,
    n=N,
)

assert (models_path / "tokenizer.json").exists()
assert rnn_path.exists()
assert (rnn_path / "assets").exists()
assert (rnn_path / "model_config.json").exists()
assert (rnn_path / "saved_model.pb").exists()
assert (rnn_path / "variables").exists()
assert EPOCHS == len(list(rnn_path.glob("*.index")))

In [ ]:
# export

def _get_tkzr_ds(out_path: Path, data_path: Path,
                 max_length: int, batch_size: int,
                 n: Optional[int]=None) -> tuple:
    """
    Function to handle the configuration of tokenizer and related datasets for training
    :param out_path: Path referencing the output directory
    :param max_length: Maximum length of sequences allowed by the models
    :batch_size: Batch size to  config. datasets
    
    :return: Tuple (tokenizer, dataset_train, dataset_validation)
    """
    
    # Load in the datasets
    # Train BPE tokenizer
    # Check if the path where the tokenizer is to be saved is not empty
    # if it is not empty then just load the tokenizer there.
    
    if (out_path / "tokenizer.json").exists():
        logging.info(f"Loading tokenizer from {str(out_path / 'tokenizer.json')}.")
        tokenizer = Tokenizer.from_file(str(out_path / "tokenizer.json"))
    else:
        logging.info(
            f"Training new tokenizer and saving to {str(out_path / 'tokenizer.json')}."
        )
        df_bpe = pd.read_json(
            data_path / "codesearchnet_java" / "bpe.jsonl", orient="records", lines=True
        )[:n]
        tokenizer = train_tokenizer(df_bpe, java_special_tokens, max_length)
        tokenizer.save(str(out_path / "tokenizer.json"), pretty=True)
        del df_bpe

    # Tokenize the dataset and convert it to tfds.
    tfds_trn_path = (
        data_path / "codesearchnet_java" / f"tfds_trn_{max_length}len_{batch_size}bs"
    )
    if tfds_trn_path.exists():
        ds_trn = tf.data.experimental.load(
            str(tfds_trn_path),
            (
                tf.TensorSpec(shape=(batch_size, max_length - 1), dtype=tf.int32),
                tf.TensorSpec(shape=(batch_size, max_length - 1), dtype=tf.int32),
            ),
        )
    else:
        df_trn = pd.read_json(
            data_path / "codesearchnet_java" / "train.jsonl",
            orient="records",
            lines=True,
        )[:n]
        ds_trn = convert_df_to_tfds(df_trn, tokenizer, max_length, batch_size)
        tfds_trn_path.mkdir(exist_ok=True)
        tf.data.experimental.save(ds_trn, str(tfds_trn_path))
        del df_trn

    tfds_val_path = (
        data_path / "codesearchnet_java" / f"tfds_val_{max_length}len_{batch_size}bs"
    )
    if tfds_val_path.exists():
        ds_val = tf.data.experimental.load(
            str(tfds_val_path),
            (
                tf.TensorSpec(shape=(batch_size, max_length - 1), dtype=tf.int32),
                tf.TensorSpec(shape=(batch_size, max_length - 1), dtype=tf.int32),
            ),
        )
    else:
        df_val = pd.read_json(
            data_path / "codesearchnet_java" / "valid.jsonl",
            orient="records",
            lines=True,
        )[:n]
        ds_val = convert_df_to_tfds(df_val, tokenizer, max_length, batch_size)
        tfds_val_path.mkdir(exist_ok=True)
        tf.data.experimental.save(ds_val, str(tfds_val_path))
        del df_val
        
    return tokenizer, ds_trn, ds_val

In [ ]:
#export

#references:
#GradientAccumulator class: https://huggingface.co/transformers/_modules/transformers/optimization_tf.html#GradientAccumulator
#use in TFTrainer: https://huggingface.co/transformers/_modules/transformers/trainer_tf.html#TFTrainer

def get_accumulation_optimizer(optimizer_class, steps = 1, **kwargs):
    """
    Creates a custom optimizer that adds gradient accumulation functionality to a tf.keras optimizer
    :param optimizer_class: Class of optimizer to add gradient accumulation to. The returned optimizer will be a subclass of this class
    :param steps: Number of gradient accumulation steps, i.e. the number of batches over which to accumulate gradients
    :param kwargs: Any other keyword arguments to be passed to the optimizer_class constructor
    :returns: AccumulationOptimizer instance that is a subclass of optimizer_class with gradient accumulation
    """
    
    class AccumulationOptimizer(optimizer_class):
        def __init__(self, steps, **kwargs):
            super().__init__(**kwargs)
            self.batch = 0
            self.steps = steps
            self.accumulator = GradientAccumulator()
            self.accumulator.reset()
            self.count = 0
            
        def apply_gradients(self, grads_and_vars, name=None, experimental_aggregate_gradients=True):
            self.count +=1
            return_value = None
            if not hasattr(self, 'vars'):
                self.vars = [gradvar[1] for gradvar in grads_and_vars]
            
            self.batch += 1
            self.accumulator([gradvar[0] for gradvar in grads_and_vars])
            
            if self.batch == self.steps:
                return_value = self.apply_accumulated_gradients(name, experimental_aggregate_gradients)
            return return_value
            
            
        def apply_accumulated_gradients(self, name=None, experimental_aggregate_gradients=True):
            if self.batch != 0:
                return_value = super().apply_gradients(zip(self.accumulator.gradients, self.vars),
                                                       name, experimental_aggregate_gradients)
                self.accumulator.reset()
                self.batch = 0
                return return_value
            
    return AccumulationOptimizer(steps, **kwargs)

In [ ]:
#export

class AccumulationCallback(tf.keras.callbacks.Callback):
    """
    Callback for use with an AccumulationOptimizer.
    Applies excess gradients at the end of an epoch when the number of batches is not divisible by the number of accumulation steps.
    """
    def __init__(self, optimizer):
        self.optimizer = optimizer
        
    def on_epoch_end(self, epoch, logs=None):
        logging.debug(self.optimizer.count)
        self.optimizer.apply_accumulated_gradients()

In [ ]:
# export

def train_tfr_hf_model(hf_model, config_class, config_dict,
                       data_path, out_path, optimizer, loss,
                       epochs=64, max_length=300,
                       batch_size=64, n=None, pretrained_model=None, gradient_accumulation = False):
    """
    Function to train a Transformer model according to the provided params
    :param hf_model: Class of the HF model to implement the Transformer
    :param config_class: HuggingFace class of the config. to apply for the model
    :param config_dict: Dictionary with the params to config. the model (ex: n_layers)
    :param data_path: Path corresponding to the data to be used
    :param out_path: Path corresponding to the path to save model-related data
    :param optimizer:
    :param loss: 
    :param epochs: Number of epochs for the training process
    :param max_length: Maximum length of the sequences processed by the model
    :param batch_size: Batch size to use
    :param n: Number of training points to take (entire ds taken by default)
    :param pretrained_model: Name/Path of the pretrained model to load
    :param gradient_accumulation: Boolean, set to True if using gradient accumulation with an AccumulationOptimizer
    """
    
    out_path.mkdir(exist_ok=True)

    tokenizer, ds_trn, ds_val = _get_tkzr_ds(out_path, data_path, max_length, batch_size, n)
    
    # Implement custom HF model
    name_sufix = None
    if pretrained_model is None:
        logging.info("Implementing model.")
        custom_config = config_class(
            vocab_size = tokenizer.get_vocab_size(),
            bos_token_id = tokenizer.token_to_id("<sos>"),
            eos_token_id = tokenizer.token_to_id("<eos>"),
            pad_token_id = tokenizer.token_to_id("<pad>"),
            **config_dict
        )
        logging.info(f"Used configuration: {custom_config}")
        hf_model_impl = hf_model(custom_config)
    else:
        logging.info(f"Loading pretrained model from: {pretrained_model}")
        name_sufix = 'pretrained'
        hf_model_impl = hf_model.from_pretrained(pretrained_model)
        
    tfr_model = TransformerHFModel(str(out_path), hf_model_impl, tokenizer, optimizer, loss, name_sufix=name_sufix)
    if gradient_accumulation:
        tfr_model.callbacks.append(AccumulationCallback(optimizer))
    
    logging.info("Starting the training of the Transformer model.")
    start_trn_time = time.time()
    tfr_model.train(ds_trn, ds_val, epochs)
    end_trn_time = time.time()
    
    logging.info(f'Finished training, Time elapsed: {end_trn_time - start_trn_time} seconds.')
    
    tfr_model.save()
    logging.info(f'Model saved.')

In [ ]:
# export

def train_tfr_keras_model(config_dict, optimizer, loss, out_path,
                          data_path, epochs=64, max_length=300,
                          batch_size=64, n=None):
    """
    Function to train Transformer models based on the miniature implementation provided
    by Keras
    
    :param config_dict:
    
    """
    out_path.mkdir(exist_ok=True)
    
    tokenizer, ds_trn, ds_val = _get_tkzr_ds(out_path, data_path, max_length, batch_size, n)
    
    config_dict['out_path'] = str(out_path)
    config_dict['max_length'] = max_length
    config_dict['batch_size'] = batch_size
    config_dict['vocab_size'] = tokenizer.get_vocab_size()
    config_dict['optimizer'] = optimizer
    config_dict['loss'] = loss
    config_dict['tokenizer'] = tokenizer
    
    min_gpt_model = MiniatureGPTModel(**config_dict)
    
    logging.info("Starting the training of the Transformer model.")
    start_trn_time = time.time()
    min_gpt_model.train(ds_trn, ds_val, epochs)
    end_trn_time = time.time()
    
    logging.info(f'Finished training, Time elapsed: {end_trn_time - start_trn_time} seconds.')
    
    min_gpt_model.save()
    logging.info(f'Model saved.')

## Sampling (Generation)

Although the implementation of models uses a common interface (class Model), the details for the generation (and training) process differ, specially for the models using HuggingFace underneath. For that reason, multiple methods are provided to perform sampling.

The storage of sample sets follows a convetion similar to the one used for storing model-related data: Each models has its own directory and its related samples are stored there, with a convetion as seen in the <i>_save_samples</i> method.

In [ ]:
# export

def _save_samples(df: pd.DataFrame, out_path: str, config_name: str, file_format:Optional[str]="jsonl"):
    """
    Function to store samples for a specific model to a jsonl file.
    :param df: Pandas Dataframe containing samples
    :param out_path: Directory path to store the csv files
    :param config_name: Model config. for handle file storage naming
    
    """
    out_path = Path(out_path)
    out_path = out_path / config_name
    out_path.mkdir(exist_ok=True)
    now = datetime.datetime.now()
    ts = datetime.datetime.timestamp(now)
    
    n = len(df)
    full_path = out_path/f"{n}-samples_{ts}.{file_format}"
    logging.info(f"Saving samples to {full_path}")
    if file_format == "jsonl":
        df.to_json(full_path, orient="records", lines=True)
    else:
        df.to_csv(full_path, index=False)

In [ ]:
# export

def perform_hf_tfr_sampling(gen_model: TransformerHFModel, n: int,
                            max_length: int, special_toks: Dict[str, str],
                            out_path: Optional[str]=None,
                            decoding_strategy: Optional[str]=None,
                            file_format: Optional[str]="jsonl",
                            max_allowed: Optional[int]=1000) -> pd.DataFrame:
    """
    Function to perform the sampling (generation process) for a given (hf) model and params.
    :param gen_model: Model instance of a loaded model to be used for generation
    :param n: Number of samples to generate
    :param special_toks: Dict containing the special tokens to replace
    :param out_path: Path to store the generated samples
    :param decoding_trategy:
    :param max_allowed: Max. number of samples to generate simultaneously (using GPU)
    
    :return: Pandas DataFrame containing the number of generated samples
    """
    logging.info('Starting sampling process')
    generated_seqs = []
    
    start_time = time.time()
    if n <= max_allowed:
        generated_seqs = gen_model.generate(n=n, max_length=max_length)
    else:
        res = n % max_allowed
        n_iterations = int(n/max_allowed)
        for i in range(n_iterations):
            seqs = gen_model.generate(n=max_allowed, max_length=max_length)
            generated_seqs += seqs
        
        if res > 0:
            seqs = gen_model.generate(n=res, max_length=max_length)
            generated_seqs += seqs
    
    gen_df = pd.DataFrame(generated_seqs, columns=["code"])
    
    # Clean "raw" generated data
    clean_df = replace_spec_toks_to_original(gen_df, special_toks)
    
    if out_path is not None:
        _save_samples(clean_df, out_path, gen_model.config_name, file_format)
    end_time = time.time()
    
    logging.info(f"Time elapsed sampling: {end_time - start_time} seconds.")
    return clean_df

In [ ]:
# export

def perform_min_tfr_model_sampling(gen_model: MiniatureGPTModel, n: int,
                                   max_length: int, special_toks: Dict[str, str],
                                   out_path: Optional[str]=None,
                                   file_format: Optional[str]="jsonl",
                                   top_k: Optional[float]=100) -> pd.DataFrame:
    """
    Function to perform sampling (generation process) for transformers (keras implementation)    
    models.
    :param:
    
    :return: pd DataFrame containing generated code snippets
    """
    
    start_time = time.time()
    gen_samples = gen_model.generate(n, top_k)
    gen_df = pd.DataFrame(gen_samples, columns=["code"])
    
    # Clean "raw" generated data
    clean_df = replace_spec_toks_to_original(gen_df, special_toks)
    
    if out_path is not None:
        _save_samples(clean_df, out_path, gen_model.model_config, file_format)
        
    end_time = time.time()
    
    logging.info(f"Time elapsed sampling: {end_time - start_time} seconds.")
    
    return clean_df

In [ ]:
# export

def perform_rnn_sampling(gen_model: RNNModel, n: int, max_length: int,
                         special_toks: Dict[str, str], temperature: Optional[float]=1.0,
                         out_path: Optional[str]=None, file_format: Optional[str]="jsonl") -> pd.DataFrame:
    """
    Function to perform sampling (generation) process for RNN models
    
    :return: pandas DataFrame containing generated code snippets
    """
    start_time = time.time()
    
    samples = []
    while(len(samples) < n):
        sample = gen_model.generate(max_length, temperature)
        samples.append(sample)
        
    gen_df = pd.DataFrame(samples, columns=["code"])
    
    # Clean "raw" generated data
    clean_df = replace_spec_toks_to_original(gen_df, special_toks)
    
    if out_path is not None:
        _save_samples(clean_df, out_path, gen_model.config_name, file_format)
        
    end_time = time.time()
    logging.info(f"Time elapsed sampling: {end_time - start_time} seconds.")
    
    return clean_df

In [11]:
# hide
from nbdev.export import notebook2script

notebook2script()

Converted 00_data.core.ipynb.
Converted 00_data.covariates.ipynb.
Converted 00_data.eda.ipynb.
Converted 01_data.transforms.ipynb.
Converted 01_model.gru.ipynb.
Converted 02_model.core.ipynb.
Converted 04_evaluation.causal.ipynb.
Converted 04_evaluation.core.ipynb.
Converted index.ipynb.
